In [1]:
# revisar codigo no estudiado

import sys

# open files
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()
f = open('labels.txt')
raw_labels = f.readlines()
f.close()


tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:   
        target_dataset.append(0

SyntaxError: unexpected EOF while parsing (<ipython-input-1-f7e445b9a367>, line 38)

In [15]:
#indentity matrices

import numpy as np

a = np.array([1,2,3])
b = np.array([0.1,0.2,0.3])
c = np.array([-1,0.5,0])
d = np.array([0,0,0])

identity_matrix = np.eye(3)
print("identity matrix")
print(identity_matrix)
print("")
print("a * identity matrix")
print(a.dot(identity_matrix))
print("")
print("b * identity matrix")
print(b.dot(identity_matrix))
print("")
print("c * identity matrix")
print(c.dot(identity_matrix))

identity matrix
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

a * identity matrix
[1. 2. 3.]

b * identity matrix
[0.1 0.2 0.3]

c * identity matrix
[-1.   0.5  0. ]


In [19]:
# forward propagation

import numpy as np

def softmax(x_):
    x = np.atleast_2d(x_)
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

#word embeddings (vocabulary data)
word_vects = {}
word_vects['yankees'] = np.array([[0.,0.,0.]])
word_vects['bears'] = np.array([[0.,0.,0.]])
word_vects['braves'] = np.array([[0.,0.,0.]])
word_vects['red'] = np.array([[0.,0.,0.]])
word_vects['sox'] = np.array([[0.,0.,0.]])
word_vects['lose'] = np.array([[0.,0.,0.]])
word_vects['defeat'] = np.array([[0.,0.,0.]])
word_vects['beat'] = np.array([[0.,0.,0.]])
word_vects['tie'] = np.array([[0.,0.,0.]])

# sentence embedding to output clasification weights
sent2output = np.random.rand(3,len(word_vects))

#transition weights 
identity = np.eye(3)

#creating a sentence embeddings
layer_0 = word_vects['red']
layer_1 = layer_0.dot(identity) + word_vects['sox']
layer_2 = layer_1.dot(identity) + word_vects['defeat']

#predict over the vocabulary
pred = softmax(layer_2.dot(sent2output))
print(pred)

[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


In [21]:
y = np.array([1,0,0,0,0,0,0,0,0])

pred_delta = pred - y
layer_2_delta = pred_delta.dot(sent2output.T)
defeat_delta = layer_2_delta * 1
layer_1_delta = layer_2_delta.dot(identity.T)
sox_delta = layer_1_delta * 1
layer_0_delta = layer_1_delta.dot(identity.T)
alpha = 0.01
word_vects['red'] -= layer_0_delta * alpha
word_vects['sox'] -= sox_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha
identity -= np.outer(layer_0,layer_1_delta) * alpha
identity -= np.outer(layer_1,layer_2_delta) * alpha
sent2output -= np.outer(layer_2,pred_delta) * alpha

In [31]:
# training a neural network for babi dataset

import sys,random,math
from collections import Counter
import numpy as np

f = open('../datasets/tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

#utils functions
def words2indices(sentence):
    idx = list()
    for word in sentence:
       idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum(axis=0)

vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)

vocab = list(vocab)

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

np.random.seed(1)
embed_size = 10
embed = (np.random.rand(len(vocab),embed_size) - 0.5) * 0.1
recurrent = np.eye(embed_size)
start = np.zeros(embed_size)
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1
one_hot = np.eye(len(vocab))

def predict(sent):
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    preds = list()
    for target_i in range(len(sent)):
        layer = {}
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))
        loss += -np.log(layer['pred'][sent[target_i]])
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) +\
                                                embed[sent[target_i]]
        layers.append(layer)
    return layers, loss

for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers,loss = predict(sent)
    
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta']\.dot(decoder.transpose 
            ())
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + \
                layers[layer_idx+1]['hidden_delta']\.dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta']\.dot(recurrent.transpose())
        
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    
    for layer_idx,layer in enumerate(layers[1:]):

        decoder -= np.outer(layers[layer_idx]['hidden'],\layer['output_delta']) * alpha / float(len(sent))

        embed_idx = sent[layer_idx]

        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * \alpha /float(len(sent))

        recurrent -= np.outer(layers[layer_idx]['hidden'],\layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))
        

SyntaxError: unexpected character after line continuation character (<ipython-input-31-661a300017cd>, line 74)